In [115]:
import numpy as np
import pandas as pd
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from kan import KAN

In [116]:

# Carregar e pré-processar dados
path_file = './../data/customer_churn_telecom_services.csv'
df = pd.read_csv(path_file)

# 1. Primeiro aplicar o OneHotEncoder nas colunas categóricas
colunas = ['PaymentMethod', 'Contract', 'InternetService']
ohe = OneHotEncoder(dtype=int, drop='if_binary')
colunas_ohe = ohe.fit_transform(df[colunas]).toarray()

# Criar DataFrame com as colunas codificadas
df_ohe = pd.DataFrame(colunas_ohe, columns=ohe.get_feature_names_out(colunas))

# 2. Concatenar com o DataFrame original (removendo as colunas originais)
data = pd.concat([df.drop(colunas, axis=1), df_ohe], axis=1)

# 3. Agora fazer as substituições nos dados combinados
replace_dict = {
    'Yes': 1,
    'No': 0,
    'Female': 1,
    'Male': 0,
    'No internet service': 0,
    'No phone service': -1
}

data.replace(replace_dict, inplace=True)

# 4. Converter todas as colunas para float32
data = data.astype(np.float32).fillna(0)

# Verificar os tipos de dados
print(data.dtypes)

# Preparar dados
X = data.drop('Churn', axis=1).values
y = data['Churn'].values


gender                                     float32
SeniorCitizen                              float32
Partner                                    float32
Dependents                                 float32
tenure                                     float32
PhoneService                               float32
MultipleLines                              float32
OnlineSecurity                             float32
OnlineBackup                               float32
DeviceProtection                           float32
TechSupport                                float32
StreamingTV                                float32
StreamingMovies                            float32
PaperlessBilling                           float32
MonthlyCharges                             float32
TotalCharges                               float32
Churn                                      float32
PaymentMethod_Bank transfer (automatic)    float32
PaymentMethod_Credit card (automatic)      float32
PaymentMethod_Electronic check 

/tmp/ipykernel_3370/3765315254.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace(replace_dict, inplace=True)


In [ ]:
def kolmogorov_smirnov(y_true, y_proba):
    # Usa a função `roc_curve` do sklearn para obter os thresholds
    fpr, tpr, thresholds = roc_curve(y_true, y_proba)
    ks_statistic = np.max(np.abs(tpr - fpr))  # KS = max(TPR - FPR)
    return ks_statistic * 100  # Retorna em escala 0-100

# Classe Dataset
class ChurnDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

def train_model(model, train_loader, optimizer, criterion, device, epochs=100):
    model.train()
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            inputs = inputs.detach()  # Isso remove gradientes!
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

def evaluate_model(model, loader, device):
    model.eval()
    y_true, y_pred, y_proba = [], [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            outputs = torch.sigmoid(model(inputs).squeeze())
            y_true.extend(labels.cpu().numpy())
            y_pred.extend((outputs > 0.5).float().cpu().numpy())
            y_proba.extend(outputs.cpu().numpy())

    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'auc_roc': roc_auc_score(y_true, y_proba),
        'ks': kolmogorov_smirnov(y_true, y_proba)
    }

def objective(trial):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Hiperparâmetros
    grid_size = trial.suggest_int('grid_size', 5, 10)
    spline_order = trial.suggest_int('spline_order', 1, 3)
    width = [X.shape[1]] + [trial.suggest_int('width_'+str(i), 1, 5) for i in range(spline_order-2)] + [1]  
    noise_scale = trial.suggest_float('noise_scale', 0.0, 0.1)
    batch_size = trial.suggest_categorical('batch_size', [16, 32])
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
  
    # KFold
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    metrics = []
     
    # Salva o vetor width como atributo do trial
    trial.set_user_attr('width', str(width))

    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        # Divisão e normalização
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

        # Datasets
        train_dataset = ChurnDataset(X_train, y_train)
        val_dataset = ChurnDataset(X_val, y_val)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        # Modelo KAN

        model = KAN(
            width=width,
            grid=grid_size,
            k=spline_order,
            noise_scale=noise_scale
        ).to(device)

        optimizer = optim.AdamW(model.parameters(), lr=lr)
        criterion = nn.BCEWithLogitsLoss()

        # Treinamento
        train_model(model, train_loader, optimizer, criterion, device)

        # Avaliação
        fold_metrics = evaluate_model(model, val_loader, device)
        metrics.append(fold_metrics)

    # Calcular médias
    avg_metrics = {k: np.mean([m[k] for m in metrics]) for k in metrics[0]}
    for k, v in avg_metrics.items():
        trial.set_user_attr(k, float(v))

    return avg_metrics['auc_roc']



In [118]:

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, show_progress_bar=True)

# Salvar resultados
results_df = study.trials_dataframe()
results_df['param_observado'] = 'Maximize AUC'

file_name = './KAN/optuna_results_kan.xlsx'
Path('./KAN/').mkdir(parents=True, exist_ok=True)

if Path(file_name).exists():
    existing_df = pd.read_excel(file_name)
    combined_df = pd.concat([existing_df, results_df], ignore_index=True).drop_duplicates()
    combined_df.to_excel(file_name, index=False)
else:
    results_df.to_excel(file_name, index=False)

print("\nMelhores hiperparâmetros:")
print(study.best_params)
print("\nMétricas médias:")
print({k: v for k, v in study.best_trial.user_attrs.items()})

[I 2025-04-13 16:01:31,116] A new study created in memory with name: no-name-5f960049-3c49-4ef3-8960-8f24ecfd80ad
  0%|          | 0/50 [00:00<?, ?it/s]

checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0


Best trial: 0. Best value: 0.850223:   2%|▏         | 1/50 [26:16<21:27:46, 1576.86s/it]

[I 2025-04-13 16:27:47,977] Trial 0 finished with value: 0.8502234666375715 and parameters: {'grid_size': 8, 'spline_order': 2, 'noise_scale': 0.08493577751399967, 'batch_size': 16, 'lr': 0.009549954779598122}. Best is trial 0 with value: 0.8502234666375715.
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0


Best trial: 0. Best value: 0.850223:   4%|▍         | 2/50 [39:57<15:05:36, 1132.02s/it]

...

[I 2025-04-14 08:33:01,883] Trial 47 finished with value: 0.848166493058551 and parameters: {'grid_size': 5, 'spline_order': 2, 'noise_scale': 0.09349897186707845, 'batch_size': 16, 'lr': 0.002232044386350678}. Best is trial 24 with value: 0.8507429848278424.
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0


Best trial: 24. Best value: 0.850743:  98%|█████████▊| 49/50 [16:40:13<14:50, 890.08s/it] 

[I 2025-04-14 08:41:44,872] Trial 48 finished with value: 0.8488357678035646 and parameters: {'grid_size': 6, 'spline_order': 2, 'noise_scale': 0.09745629471100889, 'batch_size': 32, 'lr': 0.009676773087428338}. Best is trial 24 with value: 0.8507429848278424.
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0


Best trial: 24. Best value: 0.850743: 100%|██████████| 50/50 [16:57:12<00:00, 1220.65s/it]

[I 2025-04-14 08:58:43,598] Trial 49 finished with value: 0.835661032038015 and parameters: {'grid_size': 8, 'spline_order': 1, 'noise_scale': 0.08917881885954662, 'batch_size': 16, 'lr': 2.6662364141577558e-05}. Best is trial 24 with value: 0.8507429848278424.

Melhores hiperparâmetros:
{'grid_size': 8, 'spline_order': 2, 'noise_scale': 0.04690917758405222, 'batch_size': 16, 'lr': 0.005916193252613903}

Métricas médias:
{'width': '[26, 1]', 'accuracy': 0.8047684685489008, 'precision': 0.6565692629304946, 'recall': 0.560655990462094, 'f1': 0.6039011212160893, 'auc_roc': 0.8507429848278424, 'ks': 54.65288209211627}
